In [11]:
import pandas as pd
import statsmodels.api as sm

In [3]:
df = pd.read_csv('~/Desktop/COVID19/data/200913COVID19MEXICO.csv', encoding = "ISO-8859-1")
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-09-13,1e6e4c,2,4,25,2,25,25,11,1,...,2,2,2,2,1,1,99,MÃ©xico,99,97
1,2020-09-13,10d9be,2,4,2,2,10,2,4,2,...,2,2,2,2,99,1,99,MÃ©xico,99,2
2,2020-09-13,11a9b3,2,4,9,2,9,15,58,1,...,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-09-13,077c96,2,4,9,2,16,9,5,2,...,2,2,2,2,1,1,99,MÃ©xico,99,1
4,2020-09-13,1719d0,2,4,30,1,30,30,193,1,...,2,2,2,2,99,1,99,MÃ©xico,99,97


In [6]:
def preprocess(df, drop_cols, categorical_cols):
    """Función que preprocesa los datos de COVID19 (200913COVID19MEXICO.csv)
    llevando a cabo los siguientes pasos:
    
        1. Filtrar casos positivos.
        2. Eliminar columnas que no se usarán.
        4. Reemplazar '98' con ceros ya que estos representan los nulls.
            NOTA: Excepto en la columna edad.
        5. Aplicar One Hot Encoder en las columnas categóricas.
    
    Parameters:
        df: DataFrame de origen.
        drop_cols: List, nombres de las columnas que no se usarán.
        categorical_cols: List, nombres de las columnas que representan
                        variables categóricas.
    
    Returns:
        X, y DataFrames con la información de las covariables y la variable
        respuesta respectivamente.
    """
    df = df[df['RESULTADO'] == 1]
    if 'RESULTADO' not in drop_cols: drop_cols.append('RESULTADO')
    df = df.drop(drop_cols, axis = 1)
    cols = list(df.columns)
    cols.remove('EDAD')
    for col in cols:
        df[col] = df[col].replace(98, 0)
    df = df.astype(dict(zip(categorical_cols, ['category']*len(categorical_cols))))
    df = pd.get_dummies(df)
    features = list(df.columns)
    features.remove('TIPO_PACIENTE')
    X = df[features].copy()
    y = df['TIPO_PACIENTE'].copy()
    
    return X, y

drop_cols = ['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'SECTOR', 'ENTIDAD_UM', 'INTUBADO', 'NEUMONIA', 'UCI',
    'SECTOR', 'ORIGEN', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'NACIONALIDAD',
    'HABLA_LENGUA_INDIG', 'OTRO_CASO', 'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 
    'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'EMBARAZO']
categorical_cols = ['SEXO', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION',
                    'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO']
X, y = preprocess(df, drop_cols, categorical_cols)

In [12]:
X = sm.add_constant(X)
model = OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          TIPO_PACIENTE   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     7035.
Date:                Fri, 18 Sep 2020   Prob (F-statistic):               0.00
Time:                        20:40:21   Log-Likelihood:            -3.1003e+05
No. Observations:              668381   AIC:                         6.201e+05
Df Residuals:                  668356   BIC:                         6.204e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.768e+09   5.85e+08  

In [16]:
X = X.drop(['TABAQUISMO_0', 'TABAQUISMO_1', 'TABAQUISMO_2'], axis = 1)
model = OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          TIPO_PACIENTE   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     8037.
Date:                Fri, 18 Sep 2020   Prob (F-statistic):               0.00
Time:                        20:43:39   Log-Likelihood:            -3.1005e+05
No. Observations:              668381   AIC:                         6.201e+05
Df Residuals:                  668359   BIC:                         6.204e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.664e+09   5.74e+08  